In [ ]:
from dreimac import CircularCoords

from circle_bundles.viz.pca_viz import show_pca
from circle_bundles.viz.thumb_grids import *
from circle_bundles.viz.fiber_vis import *
from circle_bundles.viz.nerve_vis import *
from circle_bundles.viz.base_vis import *
from circle_bundles.viz.lattice_vis import *
from circle_bundles.viz.circle_vis import *
from circle_bundles.bundle import build_bundle, attach_bundle_viz_methods
from circle_bundles.covers import MetricBallCover
from circle_bundles.metrics import *
from circle_bundles.local_triv import *
attach_bundle_viz_methods()

from circle_bundles.frame_reduction import *
from synthetic.nat_img_patches import *
from optical_flow.patch_viz import *
from circle_bundles.bundle_map import *
from circle_bundles.viz.bundle_dash import *

import sys
sys.path.append('/Users/bradturow/PSC')

#For saving
folder_path = '/Users/bradturow/Desktop/Diagrams/Paper Diagrams/'

# Klein Bottle Model

In [ ]:
#Generate a sampling of the Klein bottle model over RP1
n_samples = 5000
n = 3

data, base_points = sample_nat_img_kb(n_samples, n = n)
grad_dirs = get_gradient_dirs(data)
print(f'{n_samples} natural image patches generated.')

## Preliminary Analysis 

In [ ]:
#View a sample of the data
patch_vis = make_patch_visualizer()
fig = show_data_vis(data, patch_vis, sampling_method = None, max_samples = 30)
plt.show()

In [ ]:
#Show a PCA visualization of the dataset
show_pca(data)

In [ ]:
#Show an interactive visualization of the data
app = show_bundle_vis(base_points = base_points, data  = data, base_metric = RP1UnitVectorMetric())
plt.show()

In [ ]:
#Run Ripser on the dataset
from ripser import ripser
from persim import plot_diagrams
prime = 3
diagrams = ripser(data, coeff = prime, maxdim = 2, n_perm = 500)['dgms']
plot_diagrams(diagrams, show=True)    

## Bundle Analysis 

In [ ]:
#Construct a cover of the base space
n_landmarks = 12
lmk_angs = np.linspace(0, np.pi,n_landmarks, endpoint= False)
landmarks = np.array([np.cos(lmk_angs), np.sin(lmk_angs)]).T
overlap = 1.99

radius = overlap* np.pi/(2*n_landmarks)

cover = MetricBallCover(base_points, landmarks, radius, metric = RP1UnitVectorMetric())
cover_data = cover.build()

#Show a summary of the cover
summ = cover.summarize(plot=True)
plt.show()

In [ ]:
#Construct local circular coordinates and model transitions as O(2) matrices
bundle = build_bundle(
    data,
    cover,
    CircularCoords_cls=CircularCoords,
    show=True,
)


In [ ]:
#Show the correlations between local circular coordinates on overlaps
fig = bundle.compare_trivs(ncols = 4)
plt.show()

In [ ]:
#Compute class persistence on the weights filtration of the nerve
pers = bundle.get_persistence(show = True)


In [ ]:
#Construct Stiefel frames
#and show mean squared error for PSC in different dimensions

tf = bundle.get_frame_dataset(
    reducer=None,
    max_frames = None,
    subcomplex = 'max_trivial')
Phi_true = tf.Y


D = Phi_true.shape[1]
dims = list(range(2, min(D, 60) + 1, 2)) 

dims_arr, psc_err = reduction_curve_psc(
    Phi_true=Phi_true,
    U=cover.U,
    dims=dims,
    max_frames=2000,     
    rng_seed=0,
    psc_verbosity=0,
    use_manopt=False,    
    plot = True,
)


In [ ]:
#Compute a bundle map compatible with a subcomplex of the nerve
reduced_dim = 4

reducer = FrameReducerConfig(method="psc", d=reduced_dim, max_frames=1000)

bm = bundle.get_bundle_map(show_summary = True, reducer = reducer, subcomplex = 'full')
F = bm.F


In [ ]:
#Run Ripser on the dimension-reduced point cloud
from ripser import ripser
from persim import plot_diagrams
prime = 2
diagrams = ripser(F, coeff = prime, maxdim = 2, n_perm = 500)['dgms']
plot_diagrams(diagrams, show=True)    

In [ ]:
#Get a global coordinatization compatible with the maximal subcomplex of the nerve on which 
#the characteristic class representatives are coboundaries 

triv_result = bundle.get_global_trivialization()
print('Global coordinates computed.')

In [ ]:
#Show an interactive visualization of the bundle colored by fiber coordinate
fig = bundle.show_bundle(colors = triv_result.F)
plt.show()

In [ ]:
#Show a visualization of the nerve labeled with SW1

#Compute a potential for the restricted orientation class
subcomplex = bundle.get_max_trivial_subcomplex()
edges = subcomplex.kept_edges
Omega = bundle.classes.cocycle_used.restrict(edges)
phi_vec = Omega.orient_if_possible(edges)[2]
phi = {lmk: phi_vec[lmk] for lmk in range(n_landmarks)}
omega = bundle.classes.omega_O1_used

fig = bundle.show_circle_nerve(omega = omega, phi = phi)
plt.show()

In [ ]:
#Show a recovered patch diagram
per_row = 5
per_col = 9
predom_dirs = np.arctan2(base_points[:,1], base_points[:,0]) % np.pi
coords = np.column_stack([predom_dirs.reshape(-1,1), triv_result.F.reshape(-1,1)])

fig = lattice_vis(
    data,
    coords,
    patch_vis,
    per_row=per_row,
    per_col = per_col,
    figsize=15,
    thumb_px=350,   
    dpi=350         
)

plt.show()
